# 02 - Data from the Web

In [1]:
# Import libraries
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

You will need requests-futures which you can get by   
$ ``pip install requests-futures``

### Task 1.0 Retrieve the Data
Obtain the 200 top-ranking universities : name, rank, country and region, number of faculty members (international and total) and number of students (international and total)

In [2]:
# Do the request
request_url = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1507975779496'
r = requests.get(request_url)

In [3]:
univ_vals = r.json()['data']
univ_vals[0]

{'cc': 'US',
 'core_id': '410',
 'country': 'United States',
 'guide': '<a href="/where-to-study/north-america/united-states/guide" class="guide-link" target="_blank">United States</a>',
 'logo': '<img src="https://www.topuniversities.com/sites/default/files/massachusetts-institute-of-technology-mit_410_small_0.jpg" alt="Massachusetts Institute of Technology (MIT)  Logo">',
 'nid': '294850',
 'rank_display': '1',
 'region': 'North America',
 'score': '100',
 'stars': '6',
 'title': 'Massachusetts Institute of Technology (MIT)',
 'url': '/universities/massachusetts-institute-technology-mit'}

In [4]:
univ_df = pd.DataFrame(univ_vals)
univ_df.head()

,cc,core_id,country,guide,logo,nid,rank_display,region,score,stars,title,url
0,US,410,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...
1,US,573,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",297282,2,North America,98.7,5,Stanford University,/universities/stanford-university
2,US,253,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294270,3,North America,98.4,5,Harvard University,/universities/harvard-university
3,US,94,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294562,4,North America,97.7,5,California Institute of Technology (Caltech),/universities/california-institute-technology-...
4,GB,95,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294561,5,Europe,95.6,5,University of Cambridge,/universities/university-cambridge


keep only : 
1. name, rank, country and region
2. number of faculty members (international and total) and number of students (international and total)

second part must be retrieved from different request

At first I cleaned 'rank_display'. After that I found out I could just assigne the order as rank. This gives a better result since I don't have to clean 'rank_display' neither to bother with interval ranks

In [5]:
#extracts the int value from a rank formatted either as =rank, rank or 
#rank_min-rank_max (last case will be replaced by rank_min)
def clean_rank(r):
    if(r[0] == '='):
        return int(r[1:]) 
    elif len(r) > 3 and r[3] == '-':
            return int(r[:3])
    else:
        return int(r)

In [6]:
#drops useless columns and cleans ranks
univ_dr_df = univ_df.drop('guide', 1).drop('logo', 1).drop('stars', 1).drop('url', 1) \
.drop('score', 1).drop('cc', 1).drop('core_id', 1).drop('nid', 1)
univ_dr_df['rank_display'] = univ_dr_df['rank_display'].map(clean_rank)
univ_dr_df.head()

,country,rank_display,region,title
0,United States,1,North America,Massachusetts Institute of Technology (MIT)
1,United States,2,North America,Stanford University
2,United States,3,North America,Harvard University
3,United States,4,North America,California Institute of Technology (Caltech)
4,United Kingdom,5,Europe,University of Cambridge


In [7]:
#replace rank by index+1 in case this is an invalid way to do, just ignore this cell
univ_dr_df[['rank_display']] = univ_dr_df.index + 1
univ_dr_df.head()

,country,rank_display,region,title
0,United States,1,North America,Massachusetts Institute of Technology (MIT)
1,United States,2,North America,Stanford University
2,United States,3,North America,Harvard University
3,United States,4,North America,California Institute of Technology (Caltech)
4,United Kingdom,5,Europe,University of Cambridge


In [8]:
#Retrieve the soup for the caltech description page
url_caltech = 'https://www.topuniversities.com/universities/california-institute-technology-caltech#wurs'
r_caltech = requests.get(url_caltech)
soup_caltech = BeautifulSoup(r_caltech.text, 'html.parser')
soup_caltech.prettify()[0:600]

'<!DOCTYPE html>\n<html dir="ltr" version="XHTML+RDFa 1.0" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:article="http://ogp.me/ns/article#" xmlns:book="http://ogp.me/ns/book#" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:og="http://ogp.me/ns#" xmlns:product="http://ogp.me/ns/product#" xmlns:profile="http://ogp.me/ns/profile#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:schema="http://schema.org/" xmlns:sioc="http://rdfs.org/sioc/ns#" xmlns:sioct="http://rdfs.org/sioc/types#" xm'

From the soup, with a quick ctrl+f on the number displayed on the corresponding page, I was able to identify the elements which contained the value I was interested in:
1. Number of international students ``<div class="int-students-main wrapper col-md-4"> \ <div class="number">``
    
2. Number of students ``<div class="students-main wrapper col-md-4"> \ <div class="number">``

3. Number of faculty staff in total ``<div class="faculty-main wrapper col-md-4"> \ <div class="number">``

4. Number of faculty staff international ``<div class="inter faculty"> \ <div class="number">``

In [9]:
#extracts the number of international students
caltech_inter_st = soup_caltech.find('div', class_='int-students-main wrapper col-md-4').find('div', class_='number').text
caltech_inter_st[1:]

'647 '

In [10]:
#extracts the number of students
caltech_st = soup_caltech.find('div', class_ = 'students-main wrapper col-md-4').find('div', class_='number').text
caltech_st[1:]

'2,255 '

Now we can test with MIT to see whether the parsing works the same (which is probably the case). If it works, then we will iterate over the links in the dataframe to parse the values of interest for each university

In [11]:
#checks if the extraction canvas works for another university
url_mit = 'https://www.topuniversities.com/universities/massachusetts-institute-technology-mit#wurs'
r_mit = requests.get(url_mit)
soup_mit = BeautifulSoup(r_mit.text, 'html.parser')
soup_mit.find('div', class_='int-students-main wrapper col-md-4').find('div', class_='number').text[1:]


'3,717 '

In [12]:
#tries to retrieve the value contained in the child <block_p class = 'class_p'>, hence contained in <block_c class='class_c'
#returns NaN if one of both block is not found
def retrieve(soup, block_p, class_p, block_c, class_c):
    parent = soup.find(block_p, class_p)
    if parent is None:
        return 'failed'
    else:
        child = parent.find(block_c, class_c)
        if child is None:
            return 'failed'
        else:
            return int(child.text[1:].replace(',',''))
    

In [13]:
#test function for valid arguments
retrieve(soup_mit, 'div', 'int-students-main wrapper col-md-4', 'div', 'number')

3717

In [14]:
#test function for invalid argument class_p
retrieve(soup_mit, 'div', 'int-st-main wrapper col-md-4', 'div', 'number')

'failed'

In [15]:
#test function for invalid argument class_c
retrieve(soup_mit, 'div', 'int-students-main wrapper col-md-4', 'div', 'nber')

'failed'

In [16]:
#simplify the call for our usage
def retrieve_div_number(soup, class_p):
    return retrieve(soup, 'div', class_p, 'div', 'number')

In [17]:
#test function with valid arguments
retrieve_div_number(soup_mit, 'int-students-main wrapper col-md-4')

3717

In [18]:
#Prepare urls for request-loop
u_urls = np.array(univ_df[['url']].values)
univ_urls = []
main_url = 'https://www.topuniversities.com'
for url_ in u_urls:
    url = url_[0]
    univ_urls.append(main_url + url + '#wurs')
univ_urls[:5]

['https://www.topuniversities.com/universities/massachusetts-institute-technology-mit#wurs',
 'https://www.topuniversities.com/universities/stanford-university#wurs',
 'https://www.topuniversities.com/universities/harvard-university#wurs',
 'https://www.topuniversities.com/universities/california-institute-technology-caltech#wurs',
 'https://www.topuniversities.com/universities/university-cambridge#wurs']

In [19]:
len(univ_urls)

959

Since doing the 959 requests in one go didn't work, I will manually split the requests in chunks as big as possible to minimize the merge step

In [20]:
#First attempt to do the requests, too blocking for 959 requests
def get_missing_data(urls):
    international_students = []
    students = []
    international_faculty = []
    faculty = []

    for url in urls:
    
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
    
        inter_st = retrieve_div_number(soup, 'int-students-main wrapper col-md-4')
        st = retrieve_div_number(soup, 'students-main wrapper col-md-4')
        inter_fac = retrieve_div_number(soup, 'faculty-main wrapper col-md-4')
        fac = retrieve_div_number(soup, 'inter faculty')
    
        international_students.append(inter_st)
        students.append(st)
        international_faculty.append(inter_fac)
        faculty.append(fac)

    return international_students, students, international_faculty, faculty

In [21]:
#store result in arrays
inter_s, s, inter_f, f = get_missing_data(univ_urls[:5])

In [22]:
inter_s

[3717, 3611, 5266, 647, 6699]

Since this is super slow, we will use the package future-requests that will make non-blocking requests. While waiting, I will treat only a small part of the data. When requests complete, I store the results in variables and its all good. This is the reason for the small block before the function

In [23]:
#declare globals for second methods
international_st = []
students = []
international_fac = []
fac = []
bad_soups = []
futures = []
num_request = 1
count = 0

In [24]:
#helper to reset globals
def reset_arrays():
    global international_st
    global students
    global international_fac
    global fac
    global bad_soups 
    global count
    
    international_st = []
    students = []
    international_fac = []
    fac = []
    bad_soups = []
    count = 0
    return 0

In [25]:
#proceeds request when completed in background:
# extract soup, extract values
def callback(future):
    r = future.result()
    soup = BeautifulSoup(r.text, 'html.parser')
    
    global international_st
    global students
    global international_fac
    global fac
    global num_request
    global count
    global bad_soups
    count = count + 1
    if count > num_request:
        international_st = []
        students = []
        international_fac = []
        fac = []
        count = 0
    
    inter_st = retrieve_div_number(soup, 'int-students-main wrapper col-md-4')
    st = retrieve_div_number(soup, 'students-main wrapper col-md-4')
    inter_fac = retrieve_div_number(soup, 'inter faculty')
    f = retrieve_div_number(soup, 'faculty-main wrapper col-md-4')
    if(f == 'failed'):
        f = retrieve_div_number(soup, 'total faculty')
    if inter_st == 'failed' or st == 'failed' or inter_fac == 'failed' or f == 'failed':
        bad_soups.append(soup)
    international_st.append(inter_st)
    students.append(st)
    international_fac.append(inter_fac)
    fac.append(f)

Now we can do our 959 requests in background. For it to work, we must recompile the initilisation of arrays as well as the callback function

The following block will play a small sound when the requests are done. For it to work you need to install python-vlc   
$ pip install python-vlc

In [26]:
#plays a fancy sound, customizable with other sounds in the fancy_sounds folder
import vlc
def play_sound(future):
    folder_path = 'fancy_sounds/'
    music_path = 'zelda_small_item.wav'
    p = vlc.MediaPlayer(folder_path + music_path)
    p.play()
    return 0

In [27]:
#initialize session for background threads
from requests_futures.sessions import FuturesSession
session = FuturesSession()

In [28]:
#displays a progress bar for the requests in backgrounds
import threading
from IPython.display import display
import ipywidgets as widgets
import time
import sys
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

def work(progress):
    global fac
    while(len(fac) <= num_request):
        progress.value = len(fac)/num_request
thread = threading.Thread(target=work, args=(progress,))
display(progress)
thread.start()

In [29]:
#initiate background requests. The number of requests should be customized on the next line
url_slice = univ_urls[:10]
num_request = len(url_slice)
url_last_idx = len(url_slice) - 1
reset_arrays()
for i_url, url in enumerate(url_slice):
    # On fait notre requête GET
    future = session.get(url)
    # On rajoute le callback à appeler quand
    # le résultat de la requête arrive.
    if i_url != url_last_idx:
        futures.append(future)
    future.add_done_callback(callback)
    if i_url == (url_last_idx):
        future.add_done_callback(play_sound)

missing data (NaN fields):
    1. [400:500] Universidad Externado de Colombia missing international students (can put 0)
    2. [500:600] Iran University of Science and Technology missing international fac/students (can put 0)
    3. [600:700]
        1. Jadavpur University:             missing international fac (can put 0)
        2. Universidad Iberoamericana IBERO missing international fac/students (can put 0)
        3. University of Haifa              "--------------------------------------------"
        4. University of Hyderabad          missing international fac (can put 0)
        5. Anna University                  missing international fac/students (can put 0)
        6. Universidad del Pais Vasco       missing international fac (can put 0)
        7. University of Minho              missing internation fac/students (can put 0)
        8. University of Minho              "------------------------------------------"
    4. [700:800]
        1. Comenius University in Bratislava   missing international fac (can put 0)
        2. Memorial University of Newfoundland "------------------------------------"
        3. Rhodes University                   "------------------------------------"
        4. Paris Lodron University of Salzburg "------------------------------------"
        5. University of Colombo               "------------------------------------"

it seems the NaN values can be replaced by 0, I won't check the last ones since there are 15 in the next category and 13 in the last respectively [800:900] and [900:]

In [30]:
#At this point, there must be 38 bad soups when you run all requests. All bad soups values are replaced by 0 later
len(bad_soups)

0

### <b> Warning: Running the following cell will cancell all pending requests, corrupting the data in the process </b>
Note that to run the cell you have to change 'False' in 'True' in the if statement

In [31]:
# attempts to kill waiting requests: if attempt is successful, bar doesn't move. Goal is to cancel all but last requests
# in order to instantly play sound and complete only one request
if False:
    for fu in futures:
        # cancel request on this url
        fu.cancel()
    reset_arrays()
    futures = []

#Last try it seemed to work. We reset arrays twice since the canceling of
#asynchronous requests is asynchronous, the reset might happen before all requests are canceled
#and some requests might finish in the meantime (after reset)
;

''

#### <b> Next block will close background requests, uncomment and use carefully </b>

In [32]:
#bad_soups contains all the soups from which we were unable to scrap data. 
#After requests are finished, the block should compile to 0
len(bad_soups)

0

Si vous lancez de nouvelles requête avant que les précédentes soient terminées, vous avez de forte chances que la data soit corrompue. Dans ce cas attendez la fin des requêtes en cours, puis relancez la boucle ci-dessus

You can see the requests progress here. If you want to do something else, a sound will be played as the requests complete

Now that you finished your coffee and were alerted by the fancy sounds that you could pursue the compilation, we can finally use our well-deserved data

In [33]:
#construct dataframe from individual pages'info
missing_df = pd.DataFrame(data= [international_st, students, international_fac, fac],
                         index = ['inter_st', 'students', 'inter_fac', 'fac']).transpose()
missing_df.head()

,inter_st,students,inter_fac,fac
0,3717,11067,1679,2982
1,3611,15878,2042,4285


In [34]:
#merge the two daframes to have all infos needed
univ_tot_df = pd.concat([univ_dr_df, missing_df], axis = 1)
univ_rk_df = univ_tot_df.set_index(['rank_display'])
univ_rk_df.head()

,country,region,title,inter_st,students,inter_fac,fac
rank_display,,,,,,,
1,United States,North America,Massachusetts Institute of Technology (MIT),3717.0,11067.0,1679.0,2982.0
2,United States,North America,Stanford University,3611.0,15878.0,2042.0,4285.0
3,United States,North America,Harvard University,NaN,NaN,NaN,NaN
4,United States,North America,California Institute of Technology (Caltech),NaN,NaN,NaN,NaN
5,United Kingdom,Europe,University of Cambridge,NaN,NaN,NaN,NaN


### Task 1.1 
Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?

(a) ratio between faculty members and students

#### Warning
Le dataframe contient des NaN lorsque la requête n'a pas réussi à récupérer de valeur. Le choix a été fait de drop les universités qui contenaient des NaN. A discuter

In [35]:
#Add a columns in which each cell is the result of col1/col2 for corresponding cells in col1, col2
#The returned dataframe is sorted in ascending(1)/descending(0) order depending on the value of the parameter
def ratio(df, col1, col2, ascending_ = 0):
    ratio_df = df.copy().replace('failed', 0)
    ratio_df['ratio'] = ratio_df[col1] / ratio_df[col2]
    ratio_df_sorted = ratio_df.sort_values(['ratio'], ascending = ascending_)
    return ratio_df_sorted

In [36]:
ratio(univ_tot_df, 'fac', 'students', ascending_ = 0).head(5)

,country,rank_display,region,title,inter_st,students,inter_fac,fac,ratio
1,United States,2,North America,Stanford University,3611.0,15878.0,2042.0,4285.0,0.26987
0,United States,1,North America,Massachusetts Institute of Technology (MIT),3717.0,11067.0,1679.0,2982.0,0.26945
2,United States,3,North America,Harvard University,NaN,NaN,NaN,NaN,NaN
3,United States,4,North America,California Institute of Technology (Caltech),NaN,NaN,NaN,NaN,NaN
4,United Kingdom,5,Europe,University of Cambridge,NaN,NaN,NaN,NaN,NaN


The ratio of faculty/students seems correlated with the ranking

(b) ratio of international students

In [37]:
ratio(univ_tot_df, 'inter_st', 'students', ascending_ = 0).head(5)

,country,rank_display,region,title,inter_st,students,inter_fac,fac,ratio
0,United States,1,North America,Massachusetts Institute of Technology (MIT),3717.0,11067.0,1679.0,2982.0,0.335863
1,United States,2,North America,Stanford University,3611.0,15878.0,2042.0,4285.0,0.227422
2,United States,3,North America,Harvard University,NaN,NaN,NaN,NaN,NaN
3,United States,4,North America,California Institute of Technology (Caltech),NaN,NaN,NaN,NaN,NaN
4,United Kingdom,5,Europe,University of Cambridge,NaN,NaN,NaN,NaN,NaN


The ratio of international_students/students seems correlated with the ranking

(c) country

In [38]:
#Returns a multi_indexed dataframe sorted by the argument col as higher index and argument rank as second index
def sortby_col_rank(df, col):
    univ_col = df.set_index([col, 'rank_display'])
    univ_col.sort_index(level=[col, 'rank_display'], ascending=[1, 1], inplace=True)
    return univ_col

In [39]:
country_df = sortby_col_rank(univ_tot_df, 'country')
us_univ = country_df.loc['United States']
#us_univ.head(5)
country_df.head()

region  \
country   rank_display                  
Argentina 75            Latin America   
          331           Latin America   
          337           Latin America   
          364           Latin America   
          440           Latin America   

                                                            title  inter_st  \
country   rank_display                                                        
Argentina 75                    Universidad de Buenos Aires (UBA)       NaN   
          331                                 Universidad Austral       NaN   
          337                             Universidad de Belgrano       NaN   
          364           Pontificia Universidad Católica Argentina       NaN   
          440                         Universidad de Palermo (UP)       NaN   

                        students  inter_fac  fac  
country   rank_display                            
Argentina 75                 NaN        NaN  NaN  
          331                NaN        NaN  NaN  
          337                NaN        NaN  NaN  
          364                NaN        NaN  NaN  
          440                NaN        NaN  NaN

(d) region

In [40]:
region_df = sortby_col_rank(univ_tot_df, 'region')
region_df.head()

country                             title  inter_st  \
region rank_display                                                             
Africa 191           South Africa           University of Cape Town       NaN   
       362           South Africa           Stellenbosch University       NaN   
       366           South Africa   University of the Witwatersrand       NaN   
       396                  Egypt  The American University in Cairo       NaN   
       482                  Egypt                  Cairo University       NaN   

                     students  inter_fac  fac  
region rank_display                            
Africa 191                NaN        NaN  NaN  
       362                NaN        NaN  NaN  
       366                NaN        NaN  NaN  
       396                NaN        NaN  NaN  
       482                NaN        NaN  NaN

### Task 2.0 Retrieve the Data

In [41]:
request_url2 = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
r2 = requests.get(request_url2)

In [42]:
univ_vals2 = r2.json()['data']
univ_vals2[0]

{'aliases': 'University of Oxford',
 'location': 'United Kingdom',
 'member_level': '0',
 'name': 'University of Oxford',
 'nid': 468,
 'rank': '1',
 'rank_order': '10',
 'record_type': 'master_account',
 'scores_citations': '99.1',
 'scores_citations_rank': '15',
 'scores_industry_income': '63.7',
 'scores_industry_income_rank': '169',
 'scores_international_outlook': '95.0',
 'scores_international_outlook_rank': '24',
 'scores_overall': '94.3',
 'scores_overall_rank': '10',
 'scores_research': '99.5',
 'scores_research_rank': '1',
 'scores_teaching': '86.7',
 'scores_teaching_rank': '5',
 'stats_female_male_ratio': '46 : 54',
 'stats_number_students': '20,409',
 'stats_pc_intl_students': '38%',
 'stats_student_staff_ratio': '11.2',
 'subjects_offered': 'Archaeology,Art, Performing Arts & Design,Biological Sciences,Business & Management,Chemical Engineering,Chemistry,Civil Engineering,Computer Science,Economics & Econometrics,Electrical & Electronic Engineering,General Engineering,Geo

In [43]:
univ_df2 = pd.DataFrame(univ_vals2)
univ_dr_df2 = univ_df2.drop('aliases', 1).drop('member_level', 1).drop('rank_order', 1).drop('record_type', 1) \
.drop('scores_citations', 1).drop('scores_research', 1).drop('scores_research_rank', 1).drop('scores_teaching', 1) \
.drop('nid', 1).drop('scores_citations_rank', 1).drop('scores_industry_income', 1).drop('scores_industry_income_rank', 1) \
.drop('scores_overall_rank', 1).drop('scores_international_outlook', 1).drop('scores_international_outlook_rank', 1) \
.drop('scores_overall', 1).drop('scores_teaching_rank', 1).drop('stats_female_male_ratio', 1) \
.drop('stats_pc_intl_students', 1).drop('subjects_offered', 1).drop('url', 1)
univ_dr_df2.head()

,location,name,rank,stats_number_students,stats_student_staff_ratio
0,United Kingdom,University of Oxford,1,"20,409",11.2
1,United Kingdom,University of Cambridge,2,"18,389",10.9
2,United States,California Institute of Technology,=3,"2,209",6.5
3,United States,Stanford University,=3,"15,845",7.5
4,United States,Massachusetts Institute of Technology,5,"11,177",8.7


In [44]:
len(univ_dr_df2.index)

1102

In [45]:
#replace rank by index+1 in case this is an invalid way to do, just ignore this cell
univ_dr_df2['rank'] = univ_dr_df2.index + 1
univ_dr_df2.head()

,location,name,rank,stats_number_students,stats_student_staff_ratio
0,United Kingdom,University of Oxford,1,"20,409",11.2
1,United Kingdom,University of Cambridge,2,"18,389",10.9
2,United States,California Institute of Technology,3,"2,209",6.5
3,United States,Stanford University,4,"15,845",7.5
4,United States,Massachusetts Institute of Technology,5,"11,177",8.7


In [46]:
#Prepare urls for request-loop
u_urls2 = np.array(univ_df2[['url']].values)
univ_urls2 = []
main_url2 = 'http://www.timeshighereducation.com'
for url_ in u_urls2:
    url = url_[0]
    univ_urls2.append(main_url2 + url + '#wurs')
univ_urls2[:5]

['http://www.timeshighereducation.com/world-university-rankings/university-oxford#wurs',
 'http://www.timeshighereducation.com/world-university-rankings/university-cambridge#wurs',
 'http://www.timeshighereducation.com/world-university-rankings/california-institute-technology#wurs',
 'http://www.timeshighereducation.com/world-university-rankings/stanford-university#wurs',
 'http://www.timeshighereducation.com/world-university-rankings/massachusetts-institute-technology#wurs']

In [47]:
#Retrieve the soup for the caltech description page
url_oxford = 'http://www.timeshighereducation.com/world-university-rankings/university-oxford#wurs'
r_oxford = requests.get(url_oxford)
soup_oxford = BeautifulSoup(r_oxford.text, 'html.parser')
soup_oxford.prettify()[:200]

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"\n  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">\n<html dir="ltr" lang="en" xmlns:article="http://ogp.me/ns/article#" xmlns:book="http://ogp.me/'

In [48]:
oxford_values = soup_oxford.find('div', class_='panel-pane pane-data-stats') \
.find('div', class_='pane-content').findAll('div')
div_idx = 0
for i_div in range(0, len(oxford_values)):
    curr_div = oxford_values[i_div]
    if(curr_div.text == 'keystats pc_intl_students'):
        div_idx = i_div
int(oxford_values[div_idx].find('div', class_='value').text[:2])/100

0.38

In [49]:
int(oxford_values[0].text[:2])/100

0.38

In [50]:
def retrieve_div_value(soup, div_name):
    parent = soup.find('div', class_='panel-pane pane-data-stats')
    if parent is None: 
        return 'failed'
    child = parent.find('div', class_='pane-content')
    if child is None:
        return 'failed'
    values = child.findAll('div')
    if values is None:
        return 'failed'
    div_idx = 0
    last_text = ''
    for i_div in range(0, len(values)):
        curr_div = values[i_div]
        if(curr_div.text == div_name):
            return int(last_text[:len(last_text)-1])/100
        last_text = curr_div.text
    return 'failed'

In [51]:
retrieve_div_value(soup_oxford, 'Percentage of International Students')

0.38

In [52]:
#declare globals for second methods
international_st2 = []
num_request2 = 1
bad_soups2 = []
futures2 = []
count2 = 0

In [53]:
#helper to reset globals
def reset_arrays2():
    global international_st2
    global count2
    global bad_soups2
    
    international_st2 = []
    bad_soups2 = []
    count2 = 0
    return 0

In [54]:
#proceeds request when completed in background:
# extract soup, extract values
def callback2(future):
    r = future.result()
    soup = BeautifulSoup(r.text, 'html.parser')
    
    global international_st2
    global num_request2
    global count2
    global bad_soups2
    
    count2 = count2 + 1
    if count2 > num_request2:
        international_st2 = []
        count2 = 0
    
    inter_st = retrieve_div_value(soup, 'Percentage of International Students')
    #if(inter_st == 'failed'):
        
    if inter_st == 'failed':
        bad_soups2.append(soup)
    international_st2.append(inter_st)

In [55]:
#initialize session for background threads
from requests_futures.sessions import FuturesSession
session2 = FuturesSession()

In [56]:
#displays a progress bar for the requests in backgrounds
import threading
from IPython.display import display
import ipywidgets as widgets
import time
import sys
progress2 = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

def work2(progress2):
    global international_st2
    while(len(international_st2) <= num_request2):
        progress2.value = len(international_st2)/num_request2
thread2 = threading.Thread(target=work2, args=(progress2,))
display(progress2)
thread2.start()

In [57]:
len(international_st2)

0

In [58]:
#initiate background requests. The number of requests should be customized on the next line
url_slice2 = univ_urls2[:10]
num_request2 = len(url_slice2)
url_last_idx2 = len(url_slice2) - 1
reset_arrays2()
for i_url, url in enumerate(url_slice2):
    # On fait notre requête GET
    future2 = session2.get(url)
    # On rajoute le callback à appeler quand
    # le résultat de la requête arrive.
    if i_url != url_last_idx2:
        futures2.append(future)
    future2.add_done_callback(callback2)
    if i_url == (url_last_idx2):
        future2.add_done_callback(play_sound)

In [61]:
len(bad_soups2)

0

In [60]:
# attempts to kill waiting requests: if attempt is successful, bar doesn't move. Goal is to cancel all but last requests
# in order to instantly play sound and complete only one request
global futures2
if True:
    for fu in futures2:
        # cancel request on this url
        fu.cancel()
    reset_arrays2()
    futures2 = []

#Last try it seemed to work. We reset arrays twice since the canceling of
#asynchronous requests is asynchronous, the reset might happen before all requests are canceled
#and some requests might finish in the meantime (after reset)
;

''

In [62]:
#construct dataframe from individual pages'info
missing_df2 = pd.DataFrame(data= [international_st2],
                         index = ['inter_st_ratio']).transpose()
missing_df2.head()

,inter_st_ratio
0,0.38
1,0.35
2,0.27
3,0.22
4,0.34


In [79]:
#merge the two daframes to have all infos needed
univ_tot_df2 = pd.concat([univ_dr_df2, missing_df2], axis = 1)
univ_rk_df2 = univ_tot_df2.set_index(['rank'])
univ_rk_df2[['stats_number_students']].apply(lambda x: x.replace(',',''))


,stats_number_students
rank,
1,"20,409"
2,"18,389"
3,"2,209"
4,"15,845"
5,"11,177"
6,"20,326"
7,"7,955"
8,"15,857"
9,"13,525"


### Task 2.1